In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Step1: Necessery imports
import pandas as pd
import numpy as np
import matplotlib as mpl
import scipy as scipy
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from pandas import DataFrame
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Step2: Loading the data 
#Change the csv file for another brand
#Added Data types for memory optimization and faster loading
data = pd.read_csv('/kaggle/input/titanic/train.csv', error_bad_lines=False)
data.shape

In [ ]:
data.info()

***Standard deviation***

In [ ]:
std = np.std(data)
print(std)




***Coefficient of Variation***

In [ ]:
cv = np.std(data) / np.mean(data)
print(cv)

***Variance***

In [ ]:
var_full = np.var(data)
print(var_full)

# Describing the data 

In [ ]:
#Step3:Describing the data
data.describe()

In [ ]:
#Step3:Describing the data - finding the mode [most frequent]
data.mode()

In [ ]:
import pandas_profiling
report = pandas_profiling.ProfileReport(data)
from IPython.display import display
display(report)

In [ ]:
#Finding correlations

corr = data.corr()

mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr = corr.mask(mask)
fig = ff.create_annotated_heatmap(
    z=corr.to_numpy().round(2),
    x=list(corr.index.values),
    y=list(corr.columns.values),       
    xgap=3, ygap=3,
    zmin=-1, zmax=1,
colorscale='icefire',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)
fig.update_layout(title_text='Correlation Matrix (impact relationship with numbers)',
                  title_x=0.5,
                  titlefont={'size': 20},
                  width=800, height=800,
                  xaxis_showgrid=False,
                  xaxis={'side': 'bottom'},
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed',                   
                  paper_bgcolor=None,
                  template="simple_white"
                  )
fig.show()

Notice: 
1. Strong correlation between Passenger Fare & Ticket Class - People with better seating survived
2. Positive influence and impact between # of parents / children aboard the Titanic. People who were traveling with their famillies survived better. 
4. Weak negative influence in # of siblings / spouses aboard the Titanic. 
5. Conclusion: Bonds between parent child are stronger.Interesting psychological fact: Watching babies trigers empathy and active care in our subconcious mind. We can guess that people were trying to save their children first. 
6. Weak negative influence between age and survival - Younger people were not so well seated as we look on Fare-Age relationship.

# Pairplots and Linear Predictions

In [ ]:
sns.pairplot(data,palette='bright')
%time

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(data, kind="reg", plot_kws={'line_kws':{'color':'red'}})
plt.show()
%time

In [ ]:
data.info()

# Building Mathematical Model

Making Predictions
building mathematical Machine Learning Model model and making predictionsevaluating the model

Part1: Building the model

In [ ]:
#machine learning and making predictions
#Display columns
data.columns
#drop empty values
data = data.dropna(axis=0)
#selecting prediction target
y = data.Survived
#Choosing features and stroring it in X
data_features = ['Pclass','Age', 'SibSp', 'Parch', 'Fare']

X = data[data_features]
#Describe data in X
X.describe()
#Describe the head
X.head()



# Define model. Specify a number for random_state to ensure same results each run
data_model = DecisionTreeRegressor(random_state=1)

# Fit model
data_model.fit(X, y)

#Printing Predictions
print("Making predictions for the following 5 Passengers:")
print(X.head())
print("The predictions are")
print(data_model.predict(X.head()))

In [ ]:
#Evaluating and validating the model
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
data_model = DecisionTreeRegressor()
# Fit model
data_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = data_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

In [ ]:
#underfitting and overfitting
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
data_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, data_preds))

1. # To be continued... as version increments
